**IMPORTS**

In [13]:
import keras, pickle
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier

from imblearn.over_sampling import RandomOverSampler

from pathlib import Path

# Hospital patient survival

In [2]:
survival_df = pd.read_csv('datasets/final_patient_survival_reduced.csv')

In [3]:
survival_df.head()

,age,bmi,elective_surgery,ethnicity,gender,height,pre_icu_days,weight,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor,hospital_death
0,68.0,22.73,0,0.0,0.0,180.3,0.541667,73.9,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,77.0,27.42,0,0.0,1.0,160.0,0.927778,70.2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
2,25.0,31.95,0,0.0,1.0,172.7,0.000694,95.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,67.0,27.56,0,0.0,0.0,190.5,0.000694,100.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
4,59.0,57.45,0,0.0,1.0,165.1,0.000694,156.6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0


In [4]:
survival_df["hospital_death"].value_counts()

0    73471
1     7025
Name: hospital_death, dtype: int64

## Prepare data

*isolate independent and dependent variables*

In [5]:
# independent variables (for prediction)
X = survival_df.loc[:, survival_df.columns != "hospital_death"].values

# dependent variable (to predict)  
y = survival_df.loc[:, survival_df.columns == "hospital_death"].values   

In [6]:
X.shape, y.shape

((80496, 16), (80496, 1))

*split training and test data*

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

*scale features*

In [8]:
scaler = StandardScaler()
X_train[:, :] = scaler.fit_transform(X_train)
X_test[:, :] = scaler.transform(X_test)

*balance target variable by oversampling* source: https://towardsdatascience.com/how-to-balance-a-dataset-in-python-36dff9d12704

In [9]:
over_sampler = RandomOverSampler(random_state=45)
X_train_bal, y_train_bal = over_sampler.fit_resample(X_train, y_train)

## DL

**Keras model setup:** fully-connected structure with 3 layers. Using ReLU activation function on 2 layers and Sigmoid function on the 3rd layer.

*source:* https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/ (works with a dataset about diabetes among Pima women)

### Build DL model

In [17]:
# define model
model = keras.Sequential(
    [
        keras.layers.Dense(90, input_dim=16, activation='relu'),
        keras.layers.Dense(80, input_dim=16, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ]
)

**Compile model**

In [18]:
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    metrics=['accuracy']
)

**Train model**

In [19]:
model.fit(X_train_bal, y_train_bal, epochs=75, batch_size=120)

Epoch 1/75
980/980 [==============================] - 2s 2ms/step - loss: 0.6395 - accuracy: 0.6295
Epoch 2/75
980/980 [==============================] - 2s 2ms/step - loss: 0.6336 - accuracy: 0.6354
Epoch 3/75
980/980 [==============================] - 2s 2ms/step - loss: 0.6303 - accuracy: 0.6390
Epoch 4/75
980/980 [==============================] - 2s 2ms/step - loss: 0.6272 - accuracy: 0.6410
Epoch 5/75
980/980 [==============================] - 2s 2ms/step - loss: 0.6247 - accuracy: 0.6435
Epoch 6/75
980/980 [==============================] - 2s 2ms/step - loss: 0.6216 - accuracy: 0.6447
Epoch 7/75
980/980 [==============================] - 2s 2ms/step - loss: 0.6181 - accuracy: 0.6487
Epoch 8/75
980/980 [==============================] - 2s 2ms/step - loss: 0.6149 - accuracy: 0.6507
Epoch 9/75
980/980 [==============================] - 2s 2ms/step - loss: 0.6115 - accuracy: 0.6541
Epoch 10/75
980/980 [==============================] - 2s 2ms/step - loss: 0.6084 - accuracy: 0.6570

**Test model**

In [20]:
test_loss, test_acc = model.evaluate(X_test, y_test)

504/504 [==============================] - 1s 1ms/step - loss: 0.5824 - accuracy: 0.7059


**SAVE MODEL** -- https://www.tensorflow.org/guide/keras/save_and_serialize

In [21]:
model.save('models/survival_prediction.h5')

## ML

Logistic regression

In [22]:
clf = LogisticRegression(max_iter=10000)
clf.fit(
    X_train_bal,
    y_train_bal.reshape(-1,)
)

LogisticRegression(max_iter=10000)

Decision tree

In [23]:
clf_tree = DecisionTreeClassifier()
clf_tree.fit(X_train_bal, y_train_bal)

DecisionTreeClassifier()

**SAVE MODELS**

In [24]:
filepath = Path('models/survival_prediction_logistic.pkl')
pickle.dump(clf, open(filepath, "wb"))

filepath = Path('models/survival_prediction_clf_tree.pkl')
pickle.dump(clf_tree, open(filepath, "wb"))
